# Note:
- This notebook file may contain methods or algorithms that are NOT covered by the teaching content of BT4222 and hence will not be assessed in your midterm exam.
- It serves to increase your exposure in depth and breath to the practical methods in addressing the specific project topic. We believe it will be helpful for your current project and also your future internship endeavors.

## Item Based Collaborative Filtering (IBCF)

High level overview of IBCF is that we breakdown our data into two matrices

- Item to item similarity matrix $S$
- User to item affinity matrix $A$

Recommendation scores are then created by computing the matrix multiplication $A\times S$.

<img src="https://drive.google.com/uc?id=1RDVbfl8quKbGPJ3wOOwgfOFvSxFOlwIG"></img>

### Item to Item Similarity Matrix $S$

We create a $m \times m$ matrix where m is the number of item. Each element $c_{ij}$ in the matrix is the count of item $i$ and item $j$ occuring together.

Similarity matrix has the following properties:

- Symmetric, $c_{i,j} = c_{j,i}$
- Non negative: $c_{i,j} \geq 0$
- The occurrences are at least as large as the co-occurrences. I.e., the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.

We can then scale this matrix to get our final Similarity Matrix $S$ using the following techniques:

- `Jaccard`: $s_{ij}=\frac{c_{ij}}{(c_{ii}+c_{jj}-c_{ij})}$
- `lift`: $s_{ij}=\frac{c_{ij}}{(c_{ii} \times c_{jj})}$
- `counts`: $s_{ij}=c_{ij}$

Intuition on which to select?
- No scaling means that the most popular item will be recommended to the user
- Lift will recommend items that are less popular but are highly favoured by a subset of users.
- Jaccard finds a balance between the above two methods.

### User to item affinity matrix $A$


The affinity matrix in captures the strength of the relationship between each individual user and the items that user has already interacted with.

We create a $n \times m$ matrix where $n$ is the number of users and $m$ is the number of items. In thise matrix each element $a_{ij}$ tells you how much does user $i$ favour item $j$.

$$a_{ij}=\sum_k w_k \left(\frac{1}{2}\right)^{\frac{t_0-t_k}{T}} $$

where the affinity $a_{ij}$ for user $i$ and item $j$ is the weighted sum of all $k$ events involving user $i$ and item $j$. $w_k$ represents the weight of a particular event, and the power of 2 term reflects the temporally-discounted event. The $(\frac{1}{2})^n$ scaling factor causes the parameter $T$ to serve as a half-life: events $T$ units before $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### Remove seen item

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### Drawback

Algorithm itself has $O(n^3)$ complexity. Therefore it is not supposed to handle large dataset in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory.

<font size=1> Content of the notebook is taken from the following repository: https://github.com/microsoft/recommenders/tree/main/recommenders </font>

### Setting up the environment (~4mins)


In [ ]:
!sudo rm -rf /usr/local/lib/python3.8/dist-packages/OpenSSL
!sudo rm -rf /usr/local/lib/python3.8/dist-packages/pyOpenSSL-22.1.0.dist-info/

!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.5.2-0-Linux-x86_64.sh

!bash ./Miniconda3-py39_23.5.2-0-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
!pip3 install pyOpenSSL==22.0.0

!pip install recommenders[examples]

--2023-10-27 07:42:59--  https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93409434 (89M) [application/x-sh]
Saving to: ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh.2’

Miniconda3-py39_23. 100%[===================>]  89.08M   219MB/s    in 0.4s    

2023-10-27 07:43:00 (219 MB/s) - ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh.2’ saved [93409434/93409434]

PREFIX=/usr/local
Unpacking payload ...
                                                                                    
Installing base environment...



Preparing transaction: - done
Executing transaction: | done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interp

### Importing Libraries

In [ ]:
# set the environment path to find Recommenders
import sys
import logging
import scipy
import numpy as np
import pandas as pd

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, serendipity, diversity, catalog_coverage, distributional_coverage, novelty
from recommenders.models.sar import SAR

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")

### Loading the Dataset

In [ ]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

In [ ]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["UserId", "MovieId", "Rating", "Timestamp"],
    title_col="Title",
)

# Convert the float precision to 32-bit in order to reduce memory consumption
data["Rating"] = data["Rating"].astype(np.float32)

data.head()

In [ ]:
header = {
    "col_user": "UserId",
    "col_item": "MovieId",
    "col_rating": "Rating",
    "col_timestamp": "Timestamp",
    "col_prediction": "Prediction",
}

#Split the dataset into 75% train and 25% test

train, test = python_stratified_split(
    data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42
)

### Training the Model

In [ ]:
# Instantiating the model using the Jaccard Similarity method. This will find a balance between recommending popular items and obscure items.

model = SAR(
    similarity_type="jaccard",
    time_decay_coefficient=30,
    time_now=None,
    timedecay_formula=True,
    **header
)

In [ ]:
# Fitting the model on the training data and computing the matrices.
model.fit(train)

### Prediction and Evaluation

In [ ]:
# Predicting top k items for every user.
# We are not recommending items that have been rated by the user.
top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

In [ ]:
# Link movie id's to movie names.

top_k_with_titles = top_k.join(
    data[["MovieId", "Title"]].drop_duplicates().set_index("MovieId"),
    on="MovieId",
    how="inner",
).sort_values(by=["UserId", "Prediction"], ascending=False)

top_k_with_titles.head(10)

In [ ]:
# Defining arguments for calculating metrics. All ranking based metrics have the same arguments
args = [test, top_k]
kwargs = dict(
    col_user="UserId",
    col_item="MovieId",
    col_rating="Rating",
    col_prediction="Prediction",
    relevancy_method="top_k",
    k=TOP_K,
)

#### Ranking Metrics

##### MAP

It is the average precision for each user normalized over all users.

In [ ]:
eval_map = map_at_k(*args, **kwargs)
print(f"MAP: {eval_map}")

##### NDCG

Normalized Discounted Cumulative Gain (NDCG) - evaluates how well the predicted items for a user are ranked based on relevance

<font size="1"> For more information visit https://medium.com/@readsumant/understanding-ndcg-as-a-metric-for-your-recomendation-system-5cd012fb3397 <font>

In [ ]:
eval_ndcg = ndcg_at_k(*args, **kwargs)
print(f"NDCG: {eval_ndcg}")

##### Precision Recall

Precision - this measures the proportion of recommended items that are relevant

Recall - this measures the proportion of relevant items that are recommended

In [ ]:
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)
print(f"Precision: {eval_precision} \nRecall: {eval_recall}")

#### Diversity Metrics

##### Coverage


We define _catalog coverage_ as the proportion of items showing in all users’ recommendations:
$$
\textrm{CatalogCoverage} = \frac{|N_r|}{|N_t|}
$$
where $N_r$ denotes the set of items in the recommendations (`top_k` in the code below) and $N_t$ the set of items in the historical data (`train`).

_Distributional coverage_ measures how equally different items are recommended to users when a particular recommender system is used.
If  $p(i|R)$ denotes the probability that item $i$ is observed among all recommendation lists, we define distributional coverage as
$$
\textrm{DistributionalCoverage} = -\sum_{i \in N_t} p(i|R) \log_2 p(i)
$$
where
$$
p(i|R) = \frac{|M_r (i)|}{|\textrm{train}|}
$$
and $M_r (i)$ denotes the users who are recommended item $i$.


In [ ]:
cov_args = [train, top_k]
cov_kwargs = dict(
    col_user="UserId",
    col_item="MovieId",
)
cat_coverage = catalog_coverage(*cov_args, **cov_kwargs)
dist_coverage = distributional_coverage(*cov_args, **cov_kwargs)
print(f"Catalog Coverage: {cat_coverage} \nDistributional Coverage: {dist_coverage}")

##### Diversity


**Diversity**

Diversity represents the variety present in a list of recommendations.
_Intra-List Similarity_ aggregates the pairwise similarity of all items in a set. A recommendation list with groups of very similar items will score a high intra-list similarity. Lower intra-list similarity indicates higher diversity.
To measure similarity between any two items we use _cosine similarity_:
$$
\textrm{Cosine Similarity}(i,j)=  \frac{|M_t^{l(i,j)}|} {\sqrt{|M_t^{l(i)}|} \sqrt{|M_t^{l(j)}|} }
$$
where $M_t^{l(i)}$ denotes the set of users who liked item $i$ and $M_t^{l(i,j)}$ the users who liked both $i$ and $j$.
Intra-list similarity is then defined as
$$
\textrm{IL} = \frac{1}{|M|} \sum_{u \in M} \frac{1}{\binom{N_r(u)}{2}} \sum_{i,j \in N_r (u),\, i<j} \textrm{Cosine Similarity}(i,j)
$$
where $M$ is the set of users and $N_r(u)$ the set of recommendations for user $u$. Finally, diversity is defined as
$$
\textrm{diversity} = 1 - \textrm{IL}
$$


In [ ]:
div_args = [train, top_k]
div_kwargs = dict(
    col_user="UserId",
    col_item="MovieId",
)
diversity_eval = diversity(*div_args, **div_kwargs)
print(f"Diversity: {diversity_eval}")

##### Novelty


**Novelty**

The novelty of an item is inverse to its _popularity_. If $p(i)$ represents the probability that item $i$ is observed (or known, interacted with etc.) by users, then  
$$
p(i) = \frac{|M_t (i)|} {|\textrm{train}|}
$$
where $M_t (i)$ is the set of users who have interacted with item $i$ in the historical data.

The novelty of an item is then defined as
$$
\textrm{novelty}(i) = -\log_2 p(i)
$$
and the novelty of the recommendations across all users is defined as
$$
\textrm{novelty} = \sum_{i \in N_r} \frac{|M_r (i)|}{|\textrm{top_k}|} \textrm{novelty}(i)
$$


In [ ]:
nov_args = [train, top_k]
nov_kwargs = dict(
    col_user="UserId",
    col_item="MovieId",
)
novelty_eval = novelty(*nov_args, **nov_kwargs)
print(f"Novelty: {novelty_eval}")

##### Serendipity

**Serendipity**

Serendipity represents the “unusualness” or “surprise” of recommendations. Unlike novelty, serendipity encompasses the semantic content of items and can be imagined as the distance between recommended items and their expected contents (Zhang et al.) Lower cosine similarity indicates lower expectedness and higher serendipity.
We define the expectedness of an unseen item $i$ for user $u$ as the average similarity between every already seen item $j$ in the historical data and $i$:
$$
\textrm{expectedness}(i|u) = \frac{1}{|N_t (u)|} \sum_{j \in N_t (u)} \textrm{Cosine Similarity}(i,j)
$$
The serendipity of item $i$ is (1 - expectedness) multiplied by _relevance_, where relevance indicates whether the item turns out to be liked by the user or not. For example, in a binary scenario, if an item in `top_k` is liked (purchased, clicked) in `train`, its relevance equals one, otherwise it equals zero. Aggregating over all users and items, the overall
serendipity is defined as
$$
\textrm{serendipity} = \frac{1}{|M|} \sum_{u \in M_r}
\frac{1}{|N_r (u)|} \sum_{i \in N_r (u)} \big(1 - \textrm{expectedness}(i|u) \big) \, \textrm{relevance}(i)
$$


In [ ]:
ser_args = [train, top_k]
ser_kwargs = dict(
    col_user="UserId",
    col_item="MovieId",
)
ser_eval = serendipity(*ser_args, **ser_kwargs)
print(f"Serendipity: {ser_eval}")

### Summary of Ranking Metrics

<center>

|Metric|Range|Selection criteria|Limitation|
|------|-------------------------------|---------|----------|
|Precision|$\geq 0$ and $\leq 1$|Higher the better.|Only for hits in recommendations.|
|Recall|$\geq 0$ and $\leq 1$|Higher the better.|Only for hits in the ground truth.|
|NDCG|$\geq 0$ and $\leq 1$|Higher the better.|Does not penalize for bad/missing items, and does not perform for several equally good items.|
|MAP|$\geq 0$ and $\leq 1$|Higher the better.|Depend on variable distributions.|

</center>